# **Challenge**: Write a python script that downloads all Technical Drawings from https://www.heisig.com/aktuelle-auftraege/ and stores them locally.

#Project Details:
1. Allow user to  save all drawings
2. Allow user to scrape all titles and detailes for every drawing
3. save all data and export in Json and CSV files

#Future works:
1. Allow user to choose how many drawings to download
2. compute consuming time for excuation 
3. export all file names with consume time in file


Have enjoy scraping :) \
ALhasan Alshaebi\
aatl2012@gmail.com

In [3]:
from bs4 import BeautifulSoup, SoupStrainer
import requests
from urllib.parse import urlsplit
import urllib.request
import requests.exceptions
import regex as re
import csv
import pandas as pd
import os
import json
#import shutil
!pip install wget
import wget


  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=009808193cb26a90e5a7cbb6b1d6c70053ecacfc3a64ebb437db47cb1dcc329c
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [4]:
####################################################
# set header to avoid connection server error 
headers={'Accept-Encoding': 'gzip, deflate, sdch',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',}
# Crawler class to starting crawler
class Crawler:
    def __init__(self, titleTag,bodyTag):
        #self.url = url
        self.titleTag = titleTag
        self.bodyTag = bodyTag
        ########################################
    def getPage(self, url):
        try:
            req = requests.get(url,headers=headers,allow_redirects=False)
        except requests.exceptions.RequestException:
            return None        
        return BeautifulSoup(req.text, 'lxml') # return html from website
    ####################################################
    def parse(self, url):
        """
        Extract content from a given page URL
        """
        soup = self.getPage(url)
        
        drw=soup.find("div",class_="col-md-9")
        #print(drw)
        quotes=[]
        urls=[]
        # save all drawings links
        for title in drw.find_all("div",class_=self.titleTag):
          tit={}
          tit=title.get_text()
          urls.append(tit)
        print("Number of Drawings: ",len(urls))
        # extract all related information to drawings
        for ind,i in enumerate(drw.find_all("div",class_=self.bodyTag)):
          quote = {} 
          quote["Title"]=urls[ind] # get title of drawing
          quote["Content"]=i.get_text() # get details of drawing
            #print(i)
          links = ""
          #print("Donsse")
          c=0
          # extract all Links
          for link in i.findAll('a', attrs={'href': re.compile("^/Zeichnung")}):
            quote_link = {}
            c=c+1
            #quote_link["link"]=link.get('href')
            lnk="https://www.heisig.com/"+link.get('href')
            #s = urllib.request.urlopen(lnk).read().decode('utf-8')
            #file_name, headerss = urllib.request.urlretrieve(lnk)
            #with urllib.request.urlopen(url) as response, open("/content/images", 'wb') as out_file:
              #shutil.copyfileobj(response, out_file)
            #urllib.request.urlretrieve(lnk,str(c)+".tif")

            # save locally all drawings
            wget.download(lnk)
            links=links+lnk+" , "
          quote["Links"]=links
          quotes.append(quote)
          # Export all data in files 
        export_table_and_print(quotes)

In [5]:
# Here we export all extract data in files Json and CSV .
def export_table_and_print(all_data):
  # export name of file from url
  Fname = "{0.netloc}".format(urlsplit(MainPage)).replace("www.","").replace(".com","")
  # make header for out data
  table = pd.DataFrame(all_data, columns=['Title', 'Content', 'Links'])
  table.index = table.index + 1
  # Export Drawings  in json file
  if not os.path.isfile(Fname+'.json'):
    with open(Fname+'.json', 'w') as outfile:
        json.dump(all_data, outfile,indent=4)
    print("Done file created")
  else:
    # If file already exist will be appended data
    with open(Fname+'.json', 'a') as outfile:
        json.dump(all_data, outfile,indent=4)
    print("Done Data appended") 
  # Export Drawings  in csv file   
  with open(Fname+".csv", 'w', newline='') as f: 
    w = csv.DictWriter(f,['Title','Content','Links']) 
    w.writeheader() 
    for quote in all_data: 
        w.writerow(quote) 
####################################################
####################################################

In [14]:
#@markdown #Drawings Downloading:

# map main page URL
MainPage = 'https://www.heisig.com/aktuelle-auftraege' #@param {type:"string"}
# to extract more than one order
Aktuelle_auftraege = '/drehen,/fraesen-bohrwerksarbeiten' #@param {type:"string"}
#@markdown Optional:
# select all or a specific number of Drawings
Num_Drawings = 764 #@param {type:"slider", min:0, max:10000, step:1}
# set crawl all articles
All_Drawings = True #@param {type:"boolean"}
crawler = Crawler("AnfrageHL","AnfrageIN")
for order in Aktuelle_auftraege.split(","):
  crawler.parse(MainPage+order)


31
Done Data appended
13
Done Data appended
